# 資料可以從氣象開放平台去下載 但需要先去註冊

# https://opendata.cwb.gov.tw/index

# 註冊好後 會給你一個授權碼就可以查詢裡面的資料 再到 https://opendata.cwb.gov.tw/dataset/forecast/F-C0032-001

In [ ]:
import requests
import pandas as pd
import json

url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization='
data = requests.get(url)   # 取得 JSON 檔案的內容為文字
data_json = data.json()   # 轉換成 JSON 格式
location = data_json['records']['location']   # 取出 location 的內容
location


# 跟水庫水位不同的是 這邊資料因為有授權碼可以直接抓取就不須偽瀏覽器

In [ ]:
#先設置一個list之後要將for loop丟入到data
data=[]  
for i in location:
        city = i['locationName']    # 縣市名稱
        mint8 = i['weatherElement'][2]['time'][2]['parameter']['parameterName']  # 最低溫
        maxt8 = i['weatherElement'][4]['time'][1]['parameter']['parameterName']  # 最高溫
        pop8 = i['weatherElement'][1]['time'][0]['parameter']['parameterName']+' %'   # 降雨機率  
        pop = i['weatherElement'][1]['time'][0]['parameter']['parameterName']   # 降雨機率 
        data.append([city,f'{mint8+"~"+maxt8}',pop8,pop])
data

# 接下來將資料存入xlsx 先給欄位 我們這次將最高最低溫合成溫度範圍
# 再來要把台灣的行政地圖簡版 跟 weather.xlsx合併這樣就有座標

# 台灣行政地圖簡版可以從另一篇 pm25那邊獲得 

In [ ]:
columns=['縣市','溫度預估','降雨機率 %','降雨']
df=pd.DataFrame(data,columns=columns)
df.to_excel('weather.xlsx')

data1=pd.read_excel('county.xlsx')
data2=pd.read_excel('weather.xlsx')
data1 = data1.merge(data2,left_on=('縣市'),right_on=('縣市'))
data1.to_excel('weather-map.xlsx')
data1

# 在這邊要先創一個函式 用降雨機率去判斷顏色深淺
# 最後我們讀取 weather-map.xlsx 透過 folium這個套件完成繪圖
# 這個方式雖然無法做成區塊狀顯示而是用circlemarker 但速度極快
# 反之用geopandas+plotly完成的則可以更清楚 缺點是顯示速度非常久
# 必須讀取檔案較大的座標

In [ ]:
import pandas as pd
import folium,requests,re,os,json

data = pd.read_excel('weather-map.xlsx',index_col='縣市')
data =data.drop('Unnamed: 0',axis=1)

# m先設定基本的座標 預設遠近
m = folium.Map(location=(23.5,121), zoom_start=8,title="天氣預報分布圖 資料來源:氣象局")
# 地圖需要說明不然僅有顏色無法知道 這樣就不夠視覺化了
folium.map.Marker(location=(23.7,121.5),icon=folium.DivIcon(icon_size=(300,36),icon_anchor=(0,0),html="<div style='font-size:1.8em;color:black;'>天氣預報 <br>資料來源:環境資料開放平臺 <br> 20%以下綠色<br> 20-50黃色<br>50-70 紅色<br>  70>藍色 </div>")).add_to(m)
fg = folium.FeatureGroup(name="weather")
def color_producer(values):
    color="white"
    if values<=20 :
        color = 'green'
    elif values>20 and values<=50:
        color = 'yellow'
    elif values>50 and values<=70:
        color= "red"
    else:
        color= "blue"
    cr=  0.7
    return [color,cr]

for index,location in data.iterrows(): 
    values = location['降雨']
    cc = color_producer(values)
    fg.add_child(folium.CircleMarker([location['lat'],location['log']], radius = 15,tooltip=(index+'\n'+'溫度預估:'+location['溫度預估']+'\n'+'降雨機率'+location['降雨機率 %']),fill_color=cc[0],color='grey',fill_opacity=cc[1]))

m.add_child(fg)                    

# m.save('templates/weathers.html')